<a href="https://colab.research.google.com/github/luck058/yfinance-prediction/blob/main/model1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
!git clone https://github.com/luck058/yfinance-prediction

%cd yfinance-prediction

Cloning into 'yfinance-prediction'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (57/57), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 57 (delta 33), reused 3 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (57/57), 51.25 KiB | 5.69 MiB/s, done.
Resolving deltas: 100% (33/33), done.
/content/yfinance-prediction


In [3]:
%run import_data.ipynb
help()

# Set up data fetcher using:
data_fetcher = YahooFinanceDataFetcher(self, ticker_symbol, period="max", interval="1d")

# Get historical data using:
df_data = data_fetcher.get_data(verbose=False)

# Get financials using:
df_financials = data_fetcher.get_financials(verbose=False)

# Get actions using:
df_actions = data_fetcher.get_actions(verbose=False)



In [6]:
# Defined in "import_data.ipynb"
data_fetcher = YahooFinanceDataFetcher("AAPL", period="max", interval="1d")

X, y = data_fetcher.get_X_y()

display(X.head())
print(X.shape)

display(y.head())
print(y.shape)

,High,Low,Close,Volume,Dividends,Stock Splits,Open-0,Open-1,Open-2,Open-4,...,Low max_max-10,Close max_max-10,Open-0 min_min-10,High min_min-10,Low min_min-10,Close min_min-10,Open-0 mean_mean-10,High mean_mean-10,Low mean_mean-10,Close mean_mean-10
Date,,,,,,,,,,,,,,,,,,,,,
1980-12-26 00:00:00-05:00,0.122469,0.122039,0.122039,55574400,0.0,0.0,0.122039,0.111726,0.106140,0.097116,...,0.122039,0.122039,0.087232,0.087232,0.086802,0.086802,0.099952,0.100296,0.099866,0.099866
1980-12-29 00:00:00-05:00,0.124188,0.123758,0.123758,93161600,0.0,0.0,0.123758,0.122039,0.111726,0.101842,...,0.123758,0.123758,0.087232,0.087232,0.086802,0.086802,0.102444,0.102788,0.102358,0.102358
1980-12-30 00:00:00-05:00,0.121180,0.120750,0.120750,68880000,0.0,0.0,0.121180,0.123758,0.122039,0.106140,...,0.123758,0.123758,0.087232,0.087232,0.086802,0.086802,0.105151,0.105495,0.105065,0.105065
1980-12-31 00:00:00-05:00,0.117742,0.117313,0.117313,35750400,0.0,0.0,0.117742,0.121180,0.123758,0.111726,...,0.123758,0.123758,0.088951,0.089381,0.088951,0.088951,0.108202,0.108546,0.108116,0.108116
1981-01-02 00:00:00-05:00,0.119461,0.118602,0.118602,21660800,0.0,0.0,0.118602,0.117742,0.121180,0.122039,...,0.123758,0.123758,0.091530,0.091959,0.091530,0.091530,0.111167,0.111554,0.111082,0.111082


(8893, 59)


,Open
Date,
1980-12-26 00:00:00-05:00,0.122039
1980-12-29 00:00:00-05:00,0.123758
1980-12-30 00:00:00-05:00,0.121180
1980-12-31 00:00:00-05:00,0.117742
1981-01-02 00:00:00-05:00,0.118602


(8893,)


In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error, r2_score


In [9]:
def evaluate_model(model, X_train, y_train, other_scoring=[], return_estimator=False):
    cv_results = cross_validate(model, X_train, y_train, scoring=["r2", "neg_mean_squared_error"]+other_scoring, cv=10, return_estimator=return_estimator)
    print(f"R2: {print_cv_results(cv_results['test_r2'])}")
    print(f"MSE: {print_cv_results(cv_results['test_neg_mean_squared_error'] * (-1))}")
    for scoring in other_scoring:
        print(f"{scoring}: {print_cv_results(cv_results[f'test_{scoring}'])}")
    print()
    return cv_results

In [10]:
def print_cv_results(results, z=1.96):
    """Prints the mean with confidence intervals (z-score can be modified, default 95% confidence)"""
    mean = np.mean(results)
    std_err = np.std(results) / np.sqrt(len(results))
    return f"{round(mean, 2)} " + u"\u00B1" + f"{round(z * std_err, 2)}"

In [11]:
print(f"Model: LinearRegression")
linear_regression = evaluate_model(LinearRegression(), X, y, return_estimator=True)
display(pd.DataFrame(zip(X.columns, linear_regression["estimator"][0].coef_.round(3))).sort_values(by=1))


Model: LinearRegression
R2: 1.0 ±0.0
MSE: 0.0 ±0.0



,0,1
7,Open-1,-0.083
8,Open-2,-0.083
0,High,-0.000
32,Stock Splits-2,-0.000
33,Stock Splits-4,0.000
34,Stock Splits-8,0.000
35,Open-0 max_max3,-0.000
36,High max_max3,0.000
37,Low max_max3,-0.000
38,Close max_max3,-0.000


In [13]:
class PredictOpen:
    def fit(self, X, y):
        pass

    def predict(self, X):
        return X[f"Open-0"]

    def score(self, X, y):
        y_pred = self.predict(X)
        return r2_score(y, y_pred)

    def get_params(self, deep=True):
        return {}

In [15]:
print(f"Model: PredictOpen")
evaluate_model(PredictOpen(), X, y)

Model: PredictOpen
R2: 1.0 ±0.0
MSE: 0.0 ±0.0



{'fit_time': array([0.05221105, 0.02422428, 0.00821996, 0.00757909, 0.00928998,
        0.0074079 , 0.00741744, 0.00751543, 0.00815415, 0.00745487]),
 'score_time': array([0.00604177, 0.03361416, 0.00356817, 0.00389743, 0.00350595,
        0.00353765, 0.0036695 , 0.00413966, 0.00502157, 0.00340271]),
 'test_r2': array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]),
 'test_neg_mean_squared_error': array([-0., -0., -0., -0., -0., -0., -0., -0., -0., -0.])}